# Global Maps and Tabels

This notebook demonstrates preprocessing and plotting of the global maps displayed in Fig. 1 and 2, as well as Supplementary Fig. S1-S6, and the computations for the Supllementary Tables S2-S4.

### Note

To replicate the results presented in the publication, it is essential to preprocess the complete ESM output data as described in the README or Method section of the publication.

**Following steps are included in this script:**

1. **Load Period Means**:
   - Load preprocessed data for historical and future periods across multiple models and variables.
2. **Compute BGWS and Ensemble Mean Change**:
   - Calculate Blue-Green Water Share (BGWS) and ensemble mean changes.
3. **Plot Global Maps**:
   - Generate global maps to visualize spatial distributions and changes.
4. **Plot Global Maps**:
   - Generate scatter plots for comparison of different datasets and variables.
5. **Compute and Display Statistics**:
   - Compute global means and fractions for use in supplementary tables.

In [ ]:
# ========== Import Required Libraries ==========
import sys
import dask
from dask.diagnostics import ProgressBar

In [ ]:
# ========== Configure Paths ==========
# Define the full path to the directories containing utility scripts and configurations
sys.path.append('../../src')
sys.path.append('../../src/data_handling')
sys.path.append('../../src/analysis')
sys.path.append('../../src/visualization')

# Import custom utility functions and configurations
import load_data as load_dat
import process_data as pro_dat
import compute_statistics as comp_stats
import global_maps_and_tabels as glob_map_tab

#import data directory
from config import DATA_DIR

In [ ]:
# ========== Define Font Style ==========
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Nimbus Sans'

### Step 1: Load Period Means

In [ ]:
# Step 1.1: Define the datasets
experiments = ['historical', 'ssp370']
models = ['BCC-CSM2-MR', 'CESM2', 'CMCC-CM2-SR5', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'CanESM5', 
          'IPSL-CM6A-LR', 'MIROC-ES2L', 'MPI-ESM1-2-LR', 'NorESM2-MM', 'UKESM1-0-LL',
          'ERA5_land', 'OBS'
         ]
variables=['pr', 'mrro', 'tran', 'RX5day', 'vpd', 'mrso', 'lai', 'gpp', 'wue', 'evapo', 'RX5day', 'rx5day_ratio']

In [ ]:
# Step 1.2: Load the datasets
print("Loading period means...")
with ProgressBar():
    ds_dict = dask.compute(
        load_dat.load_period_mean(
            DATA_DIR, 'processed', experiments, models, variables
        )
    )[0]

### Step 2: Compute BGWS, Ensemble Mean & Std and Change

In [ ]:
# Step 2.1: # Compute BGWS and ensemble mean & std for both periods
ds_dict = pro_dat.compute_bgws(ds_dict)
ds_dict['historical'] = comp_stats.compute_ensemble_statistic(ds_dict['historical'], 'mean')
ds_dict['historical'] = comp_stats.compute_ensemble_statistic(ds_dict['historical'], 'std')
ds_dict['ssp370'] = comp_stats.compute_ensemble_statistic(ds_dict['ssp370'], 'mean')
ds_dict['ssp370'] = comp_stats.compute_ensemble_statistic(ds_dict['ssp370'], 'std')

In [ ]:
# Step 2.2: Compute change dictionaries and ensemble mean & std
ds_dict_change = pro_dat.compute_change_dict(ds_dict)
ds_dict_change['ssp370-historical'] = comp_stats.compute_ensemble_statistic(ds_dict_change['ssp370-historical'], 'mean')
ds_dict_change['ssp370-historical'] = comp_stats.compute_ensemble_statistic(ds_dict_change['ssp370-historical'], 'std')

### Step 3: Plot Global Maps (Fig. 1, 2b-c, and Supplementary Fig. S1a,c, S2, S3 and S6)

In [ ]:
# Step 3.1: Specify the period for analysis
period = 'historical'

# Options:
# - 'historical':Data for the historical period
# - 'ssp370-historical': Differences between future projections and historical period

In [ ]:
# Step 3.2: Select the variable to visualize
variable = 'bgws'

# Options include:
# - Common variables: 'pr' (precipitation), 'tran' (transpiration), 'mrro' (runoff), etc.
# - Variables for historical data only: 'gpp' (gross primary productivity), 'bgws' (Blue-Green Water Share)
# - Variables for change analysis only: 'mrso' (soil moisture change)

In [ ]:
# Step 3.3: Choose the correct dictionary for plotting based on the selected period
if period == 'ssp370-historical':
    # Use the change dictionary for differences between future and historical periods
    plotting_dict = ds_dict_change[period]
else:
    # Use the original dictionary for single-period data
    plotting_dict = ds_dict[period]

In [ ]:
# Step 3.4: Generate the global map for the selected variable and period. Select Ensemble std for Supplementary Fig. 3.
glob_map_tab.plot_var_data_on_map(plotting_dict, 'Ensemble mean', variable, period, dpi=150, filetype='jpeg',
                                  filepath="/work/ch0636/g300115/phd_project/paper_1/results/historical/maps/bgws/")

### Step 4: Plot BGWS Change Map (Fig. 2a)

In [ ]:
# Plot the BGWS regime change map. This visualizes the difference in BGWS between the historical period and the SSP3-7.0 scenario.
glob_map_tab.plot_bgws_sub_change(ds_dict['historical'], ds_dict_change['ssp370-historical'], dpi=150, filetype='jpeg',
                                  filepath=None)

### Step 5: Scatter Plots Comparison OBS, ERA5 Land and Ensemble Mean (Supplementary Fig. S1b,d)

In [ ]:
# Define reference dataset
ref='ERA5_land' #'OBS' ERA5_land
reference_dataset = ds_dict['historical'][ref]

# Compare to ensemble mean
model_dataset = ds_dict['historical']['Ensemble mean']

# Specify the variable to compare
variable_name = 'bgws'

glob_map_tab.plot_performance_scatter(reference=reference_dataset, model=model_dataset, variable=variable_name,
                                      ref_label=ref,model_label="Ensemble Mean", output_file=None) # Put path and final filename here ennding with e.g., .pdf"

### Step 6: Scatter Plots RX5day/annual (Supplementary Fig. S4)

In [ ]:
glob_map_tab.plot_scatter_comparison_rx5day_ratio(ds_dict['historical']['Ensemble mean'].rx5day_ratio.values, ds_dict['historical']['Ensemble mean'],
                                     var_name='pr', output_file=None) # Put path and final filename here ennding with e.g., .pdf"

### Step 7: Plot BGWS Switch Map (Supplementary Fig. S5)

In [ ]:
#Plot the BGWS (Blue-Green Water Share) regime shift map. This visualizes regions where the BGWS changes from blue water to green water or vice versa.
glob_map_tab.plot_bgws_flip(ds_dict, ds_dict_change, dpi=150, filetype='jpeg',
                            filepath=None)

### Step 6: Compute Global Means (Supplementary Table S2)

In [ ]:
# Compute the global mean table for selected variables. This calculates the mean values of specified variables across the entire spatial domain 
# for both historical and SSP3-7.0 scenarios, as well as their changes.
glob_mean_table = glob_map_tab.global_mean_table(ds_dict, ds_dict_change, ['bgws', 'pr', 'mrro', 'tran'], 
                                                 filepath=None)
glob_mean_table

### Step 7: Compute Global Fraction (Supplementary Table S3)

In [ ]:
# Compute the table of percentage changes in BGWS between historical and SSP3-7.0 periods based on their historical BGWS regome. 
change_table = glob_map_tab.percentage_changes_table(ds_dict['historical'], ds_dict_change['ssp370-historical'], 
                                                     filepath=None)
change_table

### Step 8: Compute Global Fraction (Supplementary Table S4)

In [ ]:
# Compute the table of BGWS regime flips (from blue water to green water or vice versa). This quantifies the spatial extent of regime shifts in the BGWS variable.
flip_table = glob_map_tab.flip_changes_table(ds_dict['historical'], ds_dict['ssp370'], 
                                             filepath=None)
flip_table